# Tiempo de permanencia de un usuario en la página


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import inline as inline

%matplotlib inline

pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

df = pd.read_csv('events.csv')
pd.options.mode.chained_assignment = None

/home/carito/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Me quedo solo con las columnas que son de interés para este análisis

In [4]:
df_tiempos=  df[['timestamp','person']]
df_tiempos['timestamp'] = pd.to_datetime(df_tiempos['timestamp'])
df_tiempos= df_tiempos.sort_values(by=["timestamp"])
df_tiempos['just_date'] = df_tiempos['timestamp'].dt.date
df_tiempos.reset_index(drop = True, inplace = True)

Agrego la columna **'diff'** que me indica _diferencia de tiempo entre los distintos eventos temporales **por usuario**_

In [5]:
df_tiempos['diff'] = df_tiempos.groupby(['person'])['timestamp'].diff()

Paso la columna a string para detectar los valores "NaT" que indican el ingreso de un nuevo usuario 

In [6]:
df_tiempos['diff'] = df_tiempos['diff'].astype(str)
df_tiempos['new_user'] = df_tiempos['diff'] =='NaT'

Agrego la columna **'diff2'** que me indica la _diferencia entre eventos temporales_ (sin importar el usuario)

In [7]:
df_tiempos['diff2'] = (df_tiempos['timestamp'] - (df_tiempos['timestamp'].shift())) / np.timedelta64(1, 'h')

Reasigno la columna **'diff'** para volver a tener datos de tipo temporal y elimina los nulos

In [8]:
df_tiempos['diff'] = df_tiempos.groupby(['person'])['timestamp'].diff()

Lleno con 0s todos los nulos del data frame

In [9]:
df_tiempos=df_tiempos.fillna(0)

Identifico en la columna **'new_session_same_user'** los _comienzos de nuevas sesiones del mismo usuario_. Para ello se tiene en cuenta un umbral de 0.48 horas. Es decir, se considera que si el tiempo entre eventos es mayor a 0.48 horas, el evento corresponde a una nueva sesion del mismo usuario. En el informe adjunto se explica detalladamente la elección de este umbral.

In [10]:
df_tiempos['new_session_same_user'] = df_tiempos['diff2'] > 0.48

Identifico en la columna **'new_session_new_user'** los comienzos de _nuevas sesiones de un usuario nuevo._ Se tiene en cuenta el mismo umbral que antes. 

In [11]:
df_tiempos["diff"]=df_tiempos["diff"]/np.timedelta64(1, 'h')
df_tiempos["new_session_new_user"] = df_tiempos["diff"] > 0.48

Para contabilizar las sesiones nuevas (que pueden corresponder al mismo usuario o a un nuevo usuario) se realiza la operacion OR entre las columnas con datos booleanos calculadas previamente.

In [12]:
df_tiempos["new_session"]=df_tiempos["new_user"]|df_tiempos["new_session_same_user"]|df_tiempos["new_session_new_user"]

Enumero las sesiones para luego separarlas por agrupación

In [13]:
df_tiempos['sessionid'] = df_tiempos['new_session'].cumsum()

Como pusimos un umbral de 0.48 horas para la finalizacion de las sesiones, elimino de la columa **'diff'** los valores que superen ese valor (Me quedo solo con los False de la columna new_sesion). Estos valores se corresponderan al primer 'diff' de cada sesion, que debería ser siempre nulo.

In [14]:
df_tiempos=df_tiempos[df_tiempos.new_session==False]

Ahora si agrupo por **sessionid** y obtengo la información buscada

In [15]:
df_tiempos

,timestamp,person,just_date,diff,new_user,diff2,new_session_same_user,new_session_new_user,new_session,sessionid
1,2018-01-01 07:32:26,9250d410,2018-01-01,0.000000,False,0.000000,False,False,False,1
2,2018-01-01 07:32:26,9250d410,2018-01-01,0.000000,False,0.000000,False,False,False,1
3,2018-01-01 07:32:26,9250d410,2018-01-01,0.000000,False,0.000000,False,False,False,1
5,2018-01-01 09:51:53,8b6dbd1d,2018-01-01,0.000556,False,0.000556,False,False,False,2
7,2018-01-01 12:40:21,8b6dbd1d,2018-01-01,0.000000,False,0.000000,False,False,False,3
8,2018-01-01 12:41:07,8b6dbd1d,2018-01-01,0.012778,False,0.012778,False,False,False,3
9,2018-01-01 12:41:34,8b6dbd1d,2018-01-01,0.007500,False,0.007500,False,False,False,3
10,2018-01-01 12:41:59,8b6dbd1d,2018-01-01,0.006944,False,0.006944,False,False,False,3
11,2018-01-01 12:42:34,8b6dbd1d,2018-01-01,0.009722,False,0.009722,False,False,False,3
12,2018-01-01 12:43:32,8b6dbd1d,2018-01-01,0.016111,False,0.016111,False,False,False,3


In [59]:
tiempos_sesiones=df_tiempos.groupby(["sessionid"]).agg({'diff':'sum','person':'first'}).reset_index()

In [60]:
tiempos_sesiones

,sessionid,diff,person
0,1,0.000000,9250d410
1,2,0.000556,8b6dbd1d
2,3,0.074722,8b6dbd1d
3,4,0.000000,8b6dbd1d
4,5,0.000000,e26651e3
5,6,0.077778,f3c1111e
6,7,0.031667,4106b1a5
7,8,0.000556,f5a3ced2
8,9,0.003056,2f7ec952
9,10,0.104444,a51a7fa8


In [65]:
session_mean = tiempos_sesiones.groupby(["person"]).agg({'diff':'mean'}).reset_index()

In [66]:
session_mean

,person,diff
0,0004b0a2,0.456389
1,0006a21a,0.020833
2,000a54b2,0.020000
3,00184bf9,0.495556
4,0019c395,0.055833
5,001bb7eb,0.151389
6,001f1653,0.000000
7,00204059,0.184074
8,0024a82b,0.068333
9,0024ad28,0.119907
